In [8]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

In [51]:
import requests
import pandas as pd
import json

url = 'http://apis.data.go.kr/5690000/sjTrafficLightInforamtion/sj_00000610'
params = {
    'serviceKey': "jLvCDpgyRWwv286cZZNgYlNM5QlnxkY/cqth3U6NFKM0me56sIRmA6IUYXaXVbBX/vsba5X+4SzuLE0PANbRcw==",
    'pageIndex': '1',
    'pageUnit': '500',
    'dataTy': 'json',
}

response = requests.get(url, params=params)

# 응답에서 body > items 만 추출
response_json = response.json()
items = response_json.get('body', {}).get('items', [])

# 필요한 컬럼만 필터링
columns_needed = [
    'roadRouteNm','roadKnd', 'la', 'lo', 'sgngnrInstlMthdSe', 'tfclghtSe',
    'tfclghtOrdr', 'tfclghtHr', 'signlHrDcsnMthdSe', 'mngInstNm',
    'telno', 'flasherOperBgngTm', 'flasherOperEndTm'
]

# DataFrame 생성 및 컬럼 선택
df = pd.DataFrame(items)
df = df[columns_needed]

# 결과 저장
df.to_csv('./dataset/traffic_lights_filtered.csv', index=False, encoding='utf-8-sig')

print("CSV 파일 생성 완료: traffic_lights_filtered.csv")


CSV 파일 생성 완료: traffic_lights_filtered.csv
